## 5.1 Image Captioning
→CV分野の中でも特に難しい分野。  
物体検知のみならず、被写体同士の関係性を考慮して、それを文章に出力する必要があるから。  
代表的なモデルはGoogleが2015年に発表した"Show and Tell"  
画像分類のために訓練されたCNNを用いて入力画像を順伝播させ、出力層の直前の特徴を抽出してRNNへの入力とし、そのRNNがキャプションを作成するというもの。  
このCNNはすでに学習済みの重みを利用しており、このように訓練されたモデルを別のタスクに利用することを転移学習と呼ぶ。  

## 5.3 大規模データセットを扱う際の注意点
まとめ  
- 大量の画像データとメタデータを整形してバイナリに固める。
- そのバイナリを逐次読み込んで別スレッドでミニバッチを作成してキューに格納するようにする。
  
今までは全てのデータセットを`placeholder`経由で入力・正解データのミニバッチの受け渡しをしてきたが、今回のように数十GBのデータセットでは全てのデータを一度にメモリにロードすることはできない。  
そのため、今までのように`placeholder`経由でやり取りをする場合には、ミニバッチはディスクからその都度データを読み込んで加工して作成する必要がある。  
しかし、そうするとこれまでは毎ステップメモリアクセスで済んでいたデータ読み込みにディスクIOが発生してしまう。  
ディスクIOはメモリアクセスの100分の1から1,000文の1程度の速度なので、これが訓練の毎ステップに発生してしまうと、どれだけハイスペックなハードを使って効率的に訓練を行ったとしても、相当な学習時間を要してしまう。  
    
これを解消するために、ディスクからデータをロードしてデータセットを生成するスレッドと訓練を行うスレッドを分割して、キューイングをうまく行うことで、訓練を行っている間にミニバッチを作成するようにするのが定石。
  
また、もう一つの工夫として、今回の例のように一つ一つは容量が小さい画像データが数十万あるようなケースだと、画像データをひとつが100MB程度になるバイナリにまとめて、それから読み込むほうが効率的。  
大量の小さい容量を逐次読み込むのは総容量が同じで1つのファイルを読み込むよりも遥かに時間がかかる。  
  
この骨が折れるような作業も、簡単かつ高速で処理できるAPIがTensorFlowには用意されている。

## 5.4 TFRecord形式によるデータのバイナリ化
本来バイナリ化は、仕様を細かく決めた上で慎重にバイト数を意識してデータのエンコード・デコードを行うことが必要だが、`TFRecord`を使えば簡単にデータの読み書きが行える。  
他の方法でもバイナリ化は可能だが、**TensorFlowは`TFRecord`を推奨しており、他の方法と比べて高速でデータの読み書きが可能である。**  
  
### Writing
`TFRecord`形式でファイルを書き込む基本的な方法として、1件のデータセットに相当する`tf.train.Example()`クラスに必要なデータを登録して、それを専用の`writer`を用いて書き込むという手順を取る。  
  
`tf.train.Features()`（複数形）はキーバリュー形式での登録を行うクラスで、`tf.train.Feature()`（単数形）は1つのバリューを定義するクラスなので注意する。  
データの型によって後者の引数が変わるので、詳しくはリファレンスか本書を参照。

In [1]:
#!/usr/bin/env python
# coding=utf-8

import glob
import tensorflow as tf


# Get a file path
IMG_LIST = [i for i in glob.glob('img/*.jpg')]

# Export as 'test.tfrecord'
with tf.python_io.TFRecordWriter('test.tfrecord') as w:
    for img in IMG_LIST:

        # 'rb' indicates to read files as binary data
        with tf.gfile.FastGFile(img, 'rb') as f:
            data = f.read()

        # Resister read bytes to key and value
        features = tf.train.Features(feature={
            'data': tf.train.Feature(bytes_list=tf.train.BytesList(
                value=[data]))
        })

        # Write into Example Class
        example = tf.train.Example(features=features)
        w.write(example.SerializeToString())

/home/tk/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Reading
`TFRecord`ファイルを読み込む方法はいくつかあり、`TFRecord`専用の`reader`も存在するが、ここではTensorFlowのDataSetAPIを使って解説を行う。  
DataSetAPIとはモデルの読み込み、加工、バッチ化、シャッフルなどの機械学習におけるデータセット周りで必須の面倒な処理を簡単に記述できる非常に強力なAPIのこと。  
  
DataSetAPIには2つの重要なクラスが存在する。  
`tf.data.Dataset()`クラスと、`tf.data.Iterator()`クラス。
  
- Dataset　・・・　データセットの集合を管理し、加工やバッチ化を行う。
- Iterator　・・・  Datasetオブジェクトをどのようにループさせるかを管理する。
  